# Experimento
## Avaliar o impacto da mudança de iterações no algoritmo de busca PSO


### Introdução
Após o primeiro passo dado no Experimento 01, é necessário encontrar um valor para o número de iterações do PSO.


### Objetivo
Este experimento tem como objetivo verificar se existem diferenças entre o número de iterações para o PSO e assim escolher um valor que tenha um ponto de alto custo/benefício entre desempenho (RMSE) e tempo de processamento. 
Serão usadas as séries de retorno das ações componentes do índice IBOVESPA.

Os número de iterações utilizados para comparação serão:
* 10.
* 30.
* 50.
* 80.
* 100.
* 150.
* 200.

### Metodologia

1. Primeiro passo é conseguir a base de dados, foi possível através do sistema do Yahoo! Finance conseguir alguns dos dados do IBOVESPA ( 43 dos 70 componentes ).

2. Transformar a série de preços em valores de retorno e criar uma matriz de padrões de entrada e saída, segundo o parâmetro de número de lags, para esse experimento foi escolhido o valor 4.

3. Para conseguir alguma relevância estatística serão feitas 50 execuções de um cross-validation para cada valor de iteração, a variável a ser analisada será o valor da raiz quadrada do erro médio quadrático (RMSE) resultante.

4. Será gerada uma matriz de 50 linhas e 7 colunas contendo todos os resultados obtidos, a primeira análise feita é se a distribuição dos valores de RMSE de cada iteração é igual, para tal foi usado um teste estatístico não-paramétrico, o **'Kruskal-Wallis H-test'**. Esse teste é o equivalente não-paramétrico do ANOVA e sua hipótese nula afirma que as distribuições dos grupos são iguais, se rejeitada é calculada uma matriz de 7 linhas e 7 colunas para verificar essas diferenças, cada elemento da matriz é o resultado de um teste não-paramétrico entre os pares de iterações 'i' e 'j', o teste é o mesmo realizado anteriormente, porém quando o número de grupos é 2 (teste pareado) ele é conhecido por **'Mann–Whitney U test'**, equivalente não paramétrico do t-test.

5. O mesmo procedimento é feito para os tempos de execução.

6. Se as matrizes forem criadas, ou seja, existe diferença estatística entre os número de iteração do PSO, é calculado um vetor com a soma de cada linha da matriz para criar um ranking desses valores em relação aos outros. 

Então cada elemento do vetor varia entre 0 e 6, onde o primeiro representa que aquele algoritmo não foi pior que nenhum outro e 6 indica que ele foi pior que todos os outros.

Esse experimento foi feito utilizando 3 diferentes regressores, para verificar se os resultados eram consistentes entre diferentes técnicas de Machine Learning, foram escolhidos o **elmK**, **elmR** e **svr**.

### Descrição

### Código - Inicialização

Importação da biblioteca **pai** ( Portfolio-AI ) e definição dos algoritmos de busca.

In [1]:
from tools import *

import pai

assets = open('ibovespa.txt', 'r').read().split('\n')
results_path = "results/"
search_function = "particle swarm"
evals = [10, 30, 50, 80, 100, 150, 200]

### Código - Run

Para realizar a busca através do método regressor.search_param() foram utilizados alguns parâmetros extras:
* Tipo do cross-validation: time series cross-validation
* Número de folds do cross-validation: 10
* Algoritmo de busca: PSO
* Função objetivo a ser otimizada: RMSE


In [2]:
def run(name):

    r = pai.Regressor(name)

    if not os.path.exists(results_path):
        os.makedirs(results_path)

    for i, asset in enumerate(assets):

        filename = results_path + name + "_" + asset + ".p"

        if check(filename):
            try:
                stock = get_data(asset)
                stock.create_database(4, series_type="return")
                data = stock.get_database()

                result = {"finished": False}

                for ev in evals:
                    result[ev] = {}

                    print(asset, ev)

                    metrics = []
                    time = []
                    for it in range(50):
                        start = datetime.datetime.now()
                        r.search_param(database=data, cv="ts", cv_nfolds=10,
                                       of="rmse", opt_f=search_function, eval=ev,
                                       print_log=False, kf=["rbf"], f=["sigmoid"])
                        end = datetime.datetime.now()
                        delta = end - start

                        time.append(delta.total_seconds())
                        metrics.append(r.regressor.cv_best_error)

                    result[ev]["cv"] = metrics
                    result[ev]["cv_mean"] = np.mean(metrics)
                    result[ev]["cv_std"] = np.std(metrics)

                    result[ev]["time"] = time
                    result[ev]["time_mean"] = np.mean(time)
                    result[ev]["time_std"] = np.std(time)

                cvs = [result[ev]["cv"] for ev in evals]
                ts = [result[ev]["time"] for ev in evals]

                # Create paired test matrices only if exists differences among
                # sample distributions
                if not non_parametric_check_samples(cvs):
                    cv_nparam = matrix_non_parametric_paired_test(cvs)
                    result["cv_nparam"] = cv_nparam

                if not non_parametric_check_samples(ts):
                    t_nparam = matrix_non_parametric_paired_test(ts)
                    result["t_nparam"] = t_nparam

                result["finished"] = True
                pickle.dump(result, open(filename, "wb"))

                import pprint
                pprint.pprint(result)

            except:
                print("Error: ", asset)


### Código - Analysis

In [3]:
def analysis(name):
    time_means_mx = []
    cv_means_mx = []
    ranks_mx = []

    for asset in assets:
        filename = results_path + name + "_" + asset + ".p"

        try:
            result = pickle.load(open(filename, 'rb'))

            if result["finished"]:
                print(name, "Asset: ", asset)
                # print(result["cv_nparam"])
                # print()

                if "cv_nparam" not in result:
                    print()
                    for i in evals:
                        print(i)
                        print(result[i]["cv"])

                    tests = [result[sf]["cv"] for sf in evals]
                    print(non_parametric_check_samples(tests))
                    test_nparam = matrix_non_parametric_paired_test(tests)
                    print("cv_nparam")
                    print(test_nparam)


                ranks = []
                time_means = []
                cv_means = []
                for i in range(result["cv_nparam"].shape[0]):
                    cv_means.append(result[evals[i]]["cv_mean"])
                    time_means.append(result[evals[i]]["time_mean"])
                    ranks.append(np.sum(result["cv_nparam"][i, :]))
                    # sum = np.sum(result["cv_nparam"][i, :])
                    # print(evals[i], " is worst than ", sum, " functions")
                # print()

                cv_means_mx.append(cv_means)
                time_means_mx.append(time_means)
                ranks_mx.append(ranks)

        except:
            pass
            # print("Error: ", asset)
            # print()

    cv = np.array(cv_means_mx)
    ts = np.array(time_means_mx)
    rk = np.array(ranks_mx)

    cv_mean, cv_std = mean_std(cv, "cv")
    ts_mean, ts_std = mean_std(ts, "ts")
    rk_mean, rk_std = mean_std(rk, "rk")
    print()
    print("Evaluations | Ranking Sums")
    for j, ev in enumerate(evals):print(ev, ": ", np.sum(rk[:, j]))
    print()

    rk_samples = [rk[:, j] for j in range(rk.shape[1])]
    print("Rankings have same distribution: ",
          non_parametric_check_samples(rk_samples))
    rk_final = matrix_non_parametric_paired_test(rk_samples)
    # print("Rk final:")
    # print(rk_final)
    # print()

    print(rk.shape)
    print("(Evaluations | Rank | Rank Mean | Time Mean)")
    rank = [(evals[i], np.sum(rk_final[i, :]), rk_mean[i], ts_mean[i]) for i in range(rk_final.shape[0])]
    rank = sorted(rank, key=lambda f: f[1] + f[2])
    print()
    for r in rank:print(r)


### Código - Run elmK

Rodando a análise dos dados gerados pelo regressor **elmK**.

In [4]:
analysis("elmk")

elmk Asset:  ABEV3.SA
elmk Asset:  BBAS3.SA
elmk Asset:  BBDC3.SA
elmk Asset:  BBSE3.SA
elmk Asset:  BISA3.SA
elmk Asset:  BRKM5.SA
elmk Asset:  BRML3.SA
elmk Asset:  BRPR3.SA
elmk Asset:  BVMF3.SA
elmk Asset:  CCRO3.SA
elmk Asset:  CESP6.SA
elmk Asset:  CIEL3.SA
elmk Asset:  CMIG4.SA
elmk Asset:  CPFE3.SA
elmk Asset:  CPLE6.SA
elmk Asset:  CRUZ3.SA
elmk Asset:  CSAN3.SA
elmk Asset:  CYRE3.SA
elmk Asset:  DTEX3.SA
elmk Asset:  ECOR3.SA
elmk Asset:  ELET3.SA
elmk Asset:  ELET6.SA
elmk Asset:  ELPL4.SA
elmk Asset:  EVEN3.SA
elmk Asset:  ITUB4.SA
elmk Asset:  LIGT3.SA
elmk Asset:  MMXM3.SA
elmk Asset:  MRFG3.SA
elmk Asset:  MRVE3.SA
elmk Asset:  NATU3.SA
elmk Asset:  OIBR4.SA
elmk Asset:  PDGR3.SA
elmk Asset:  PETR3.SA
elmk Asset:  PETR4.SA
elmk Asset:  QUAL3.SA
elmk Asset:  RSID3.SA
elmk Asset:  SBSP3.SA
elmk Asset:  SUZB5.SA
elmk Asset:  TBLE3.SA
elmk Asset:  TIMP3.SA
elmk Asset:  VALE3.SA
elmk Asset:  VALE5.SA
elmk Asset:  VIVT4.SA

Evaluations | Ranking Sums
10 :  246.0
30 :  190.0
50

### Resultado - elmK

####1.

Evaluations | Ranking Sums |
-----------------|--------------|
10 |  246.0
30 |  190.0
50 |  142.0
80 |  67.0
100 |  45.0
150 |  27.0
200 |  0.0

O resultado do elmK mostra o que era esperado, quanto maior o número de iterações, melhor o resultado alcançado.

Além disso é feita mais uma checagem para dar uma força estatística ao resultado, é criada uma matriz contendo o vetor de rankings de cada uma das séries verificadas, formando assim uma matriz de 43 linhas e 7 colunas.

####2.

É feito um teste não-paramétrico para verificar se os rankings apresentam uma mesma distribuição.

>Rankings have same distribution:  **False**

A hipótese nula foi negada, então é criada uma matriz quadrada contendo os resultados dos testes não-paramétricos pareados. A partir dela, gera-se um vetor final contendo o  ranking.

####3.

Com isso temos o resultado final:

Evaluations | Rank | Rank Mean | Time Mean
------------------|----|----------|------------
200 | 0.0 | 0.0 | 28.985874090232553
150 | 1.0 | 0.62790697674418605| 8.0852170162790706
100 | 2.0 | 1.0465116279069768| 5.3256890544186044
80| 3.0 | 1.558139534883721| 4.2537387520930245
50|  4.0| 3.3023255813953489| 2.5558480488372095
30|  5.0| 4.4186046511627906| 1.5598874967441858
10|  6.0| 5.7209302325581399| 0.56767122976744189


Pelo resultado final fica evidente que quanto maior o número de iterações, melhor!

### Resultado - elmR

####4.

Realizando a análise nos dados gerados pelo elmR:

Evaluations | Ranking Sums |
-----------------|--------------|
10 |  258.0
30 |  207.0
50 |  155.0
80 |  73.0
100 |  42.0
150 |  20.0
200 |  11.0

>Rankings have same distribution:  **False**

Evaluations | Rank | Rank Mean | Time Mean
------------------|----|----------|------------
200 | 0.0 | 0.2558139534883721| 22.978089282790702
150 | 0.0 | 0.46511627906976744| 17.809725481395347
100 | 2.0 | 0.97674418604651159| 11.770818281860464
80| 3.0 | 1.6976744186046511| 9.3961927283720943
50|  4.0| 3.6046511627906979| 5.8645047386046514
30|  5.0| 4.8139534883720927| 3.6145857646511628
10|  6.0| 6.0| 1.3012885176744184

### Resultado - svr
####5.

Realizando a análise nos dados gerados pelo svr:

Evaluations | Ranking Sums |
-----------------|--------------|
10 |  166.0
30 |  102.0
50 |  64.0
80 |  48.0
100 |  45.0
150 |  27.0
200 |  0.0

>Rankings have same distribution:  **False**

Evaluations | Rank | Rank Mean | Time Mean
------------------|----|----------|------------
200 | 0.0 | 0.0| 15.064930754482758
150 | 1.0 | 0.93103448275862066| 11.067770089655168
100 | 2.0 | 1.5517241379310345| 7.8917772793103431
80| 2.0 | 1.6551724137931034| 5.4089081331034476
50|  4.0| 2.2068965517241379| 3.6380196351724141
30|  5.0| 3.5172413793103448| 2.9917879227586202
10|  6.0| 5.7241379310344831| 2.8240568124137928


### Conclusão

Os resultados têm uma tendência a melhorarem a medida que o valor comparado cresce, então 

Para a continuação dos experimentos, usarei o valor 100.